In [1]:
import sqlite3
import pandas as pd
import numpy as np
import urllib
import json

In [2]:
# 시 코드 & SGG → 각 도시별 시군구 데이터 
# SGG, EMD
def getAddInfo(code, types):
    res = urllib.request.urlopen("http://www.k-apt.go.kr/cmmn/bjd/getBjdList.do?bjd_code={}&bjd_gbn={}".format(code,types))
    data = json.loads(res.read().decode('utf-8'))
    df = pd.DataFrame(data['resultList'])
    df['CODE2'] = str(code)
    return df

# 아파트코드마다 아파트 데이터 얻기
def getAptInfo(kapt_code):
    res = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code={}".format(kapt_code))
    data = json.loads(res.read().decode('utf-8'))
    return data

In [3]:
conn = sqlite3.connect("k-apt.db")
cur = conn.cursor()

In [ ]:
# 테이블 삭제
#cur.execute("drop table 도시")
#cur.execute("drop table 시군구")
#cur.execute("drop table 동")
#cur.execute("drop table 아파트")

In [ ]:
# 테이블 생성
cur.execute("create table 도시(도시코드, 도시명)")
cur.execute("create table 시군구(시군구코드, 시군구명, 도시코드, 도시시군구코드)")
cur.execute("create table 동(동코드, 동명, 도시시군구코드, 도시시군구동코드)")
cur.execute("create table 아파트(아파트코드, 도시시군구동코드)")

In [ ]:
# 도시코드 테이블 적재
data = {'도시코드':[11,26,27,28,29,30,31,36,41,42,43,44,45,46,47,48,50],
        '도시명':['서울특별시','부산광역시','대구광역시','인천광역시','광주광역시','대전광역시','울산광역시','세종특별자치시','경기도','강원도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도','제주특별자치도']}
cur.executemany("insert into 도시 values (?,?)", list(zip(data['도시코드'],data['도시명'])))
conn.commit()

In [ ]:
# 시군구코드 테이블 적재
cur.execute("select 도시코드 from 도시")
rows = cur.fetchall()
codes = list(map(lambda x:x[0],rows))
for cd in codes:
    data = getAddInfo(cd,'SGG')
    data['CODE12'] = data['CODE2'].astype(str) + data['CODE'].astype(str)
    cur.executemany("insert into 시군구 values (?,?,?,?)", list(zip(data['CODE'],data['CODE_VALUE'],data['CODE2'],data['CODE12'])))
conn.commit()

In [ ]:
# 동코드 테이블 적재
cur.execute("select 도시시군구코드 from 시군구")
rows = cur.fetchall()
codes = list(map(lambda x:x[0],rows))
for cd in codes:
    data = getAddInfo(cd,'EMD')
    data['CODE12'] = data['CODE2'].astype(str) + data['CODE'].astype(str)
    cur.executemany("insert into 동 values (?,?,?,?)",list(zip(data['CODE'],data['CODE_VALUE'],data['CODE2'],data['CODE12'])))
conn.commit()

In [ ]:
# 아파트코드 테이블 적재
cur.execute("select 도시시군구동코드 from 동 where 도시시군구동코드 not in (select distinct 도시시군구동코드 from 아파트)")
rows = cur.fetchall()
codes = list(map(lambda x:x[0],rows))
for cd in codes:
    res = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptList.do?bjd_code={}".format(cd))
    data = json.loads(res.read().decode('utf-8'))
    print(list(map(lambda x:(x['KAPT_CODE'],cd),data['resultList'])))
    cur.executemany("insert into 아파트 values (?,?)",list(map(lambda x:(x['KAPT_CODE'],cd),data['resultList'])))
    conn.commit()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('A11052201', '11110171')]
[]
[]
[]
[('A11081503', '11110187'), ('A11008001', '11110187'), ('A10027708', '11110187'), ('A11081302', '11110187')]
[]
[]
[]
[('A11005401', '11110115')]
[]
[]
[]
[]
[]
[]
[]
[('A11087601', '11110175'), ('A11087602', '11110175'), ('A11055002', '11110175')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('A11034001', '11110133')]
[]
[('A11041001', '11110157')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('A11054101', '11110174'), ('A11054301', '11110174'), ('A11077101', '11110174')]
[]
[]
[]
[]
[]
[]
[]
[]
[('A11084601', '11110183'), ('A11001203', '11110183')]
[]
[('A10026946', '11110181')]
[]
[]
[]
[]
[]
[('A10027162', '11110162')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('A10080701', '11140173')]
[(